## Graph Convolutions For SIDER

In [ ]:
Featurization=ConvMolFeaturizer

É utilizado com modelos GraphConvModel.

Alternativamente, também implementamos este modelo (GraphConvModel) usando layers do TensorGraph

In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import tensorflow as tf
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvModel


/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Load SIDER Dataset

All MoleculeNet datasets are split into training, validation and test subsets following a 80/10/10 ratio. 

Different  splittings are recommended depending on each dataset's contents.

In [3]:
sider_tasks, sider_datasets, transformers = dc.molnet.load_sider(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = sider_datasets

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [4]:
len(sider_tasks)

27

In [5]:
sider_tasks

['Hepatobiliary disorders',
 'Metabolism and nutrition disorders',
 'Product issues',
 'Eye disorders',
 'Investigations',
 'Musculoskeletal and connective tissue disorders',
 'Gastrointestinal disorders',
 'Social circumstances',
 'Immune system disorders',
 'Reproductive system and breast disorders',
 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)',
 'General disorders and administration site conditions',
 'Endocrine disorders',
 'Surgical and medical procedures',
 'Vascular disorders',
 'Blood and lymphatic system disorders',
 'Skin and subcutaneous tissue disorders',
 'Congenital, familial and genetic disorders',
 'Infections and infestations',
 'Respiratory, thoracic and mediastinal disorders',
 'Psychiatric disorders',
 'Renal and urinary disorders',
 'Pregnancy, puerperium and perinatal conditions',
 'Ear and labyrinth disorders',
 'Cardiac disorders',
 'Nervous system disorders',
 'Injury, poisoning and procedural complications']

Different classification and regress metrics are recommended based on previous works and dataset's contents:
          ROC-AUC:  Area Under Curve of Receiver Operating Characteristics
          PRC-AUC:  Area Under Curve of Precision Recall Curve
          RMSE: Root-Mean-Square Error
          MAE: Mean Absolute Error

In [6]:
num_epochs = 10
losses = []

model = GraphConvModel(
    len(sider_tasks), batch_size=50, mode='classification')

for i in range(num_epochs):
    # Set nb_epoch=10 for better results.
    loss = model.fit(train_dataset, nb_epoch=1)
    print("Epoch %d loss: %f" % (i, loss))
    losses.append(loss)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 0 loss: 828.177108
Epoch 1 loss: 802.151752
Epoch 2 loss: 789.594849
Epoch 3 loss: 770.671883
Epoch 4 loss: 759.342407
Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


Epoch 5 loss: 759.136705
Epoch 6 loss: 744.528864
Epoch 7 loss: 728.219039
Epoch 8 loss: 719.593187
Epoch 9 loss: 712.222601


In [7]:
import matplotlib.pyplot as plot

plot.figure(figsize=(10,7))
plot.ylabel("Loss")
plot.xlabel("Epoch")
x = range(num_epochs)
y = losses
plot.scatter(x, y)
plot

<module 'matplotlib.pyplot' from '/home/mamonteiro/anaconda3/envs/lei/lib/python3.6/site-packages/matplotlib/pyplot.py'>

In [8]:
metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
print("Training ROC-AUC Score: %f" % train_scores["mean-roc_auc_score"])


Evaluating model
computed_metrics: [0.7454158964879851, 0.7048954733359369, 0.9643962848297214, 0.7099743588053444, 0.7353541969917516, 0.7097031772878722, 0.7870208805909382, 0.7646466928147524, 0.6464145972959361, 0.7595839177185599, 0.7432386853870891, 0.7108224877320247, 0.7510797363170258, 0.7730334432406971, 0.7032080718057405, 0.7550073460578323, 0.7766246602421547, 0.7144959566025416, 0.6989220842191639, 0.7013563776025873, 0.7133301298919219, 0.733575789930266, 0.7964975179260894, 0.7014757602074442, 0.7197216714401116, 0.8108054226475279, 0.6457487221991988]
Training ROC-AUC Score: 0.739865


In [9]:
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
print("Validation ROC-AUC Score: %f" % valid_scores["mean-roc_auc_score"])

computed_metrics: [0.7023338607594937, 0.5304101838755304, 0.5140845070422535, 0.5763694522191123, 0.6645379413974455, 0.4977443609022556, 0.6406423034330011, 0.6132203389830508, 0.5398936170212766, 0.5848091302636758, 0.5554617117117118, 0.38154269972451793, 0.6225490196078431, 0.6495934959349594, 0.5693215339233039, 0.6304347826086956, 0.6387795275590551, 0.6612021857923498, 0.5718604651162791, 0.5658391797085807, 0.5448837209302326, 0.6137646547980895, 0.7175572519083969, 0.605875152998776, 0.6337965038099507, 0.6895674300254453, 0.5127877237851662]
Validation ROC-AUC Score: 0.593662


In [10]:
num_epochs = 15
losses = []

model = GraphConvModel(
    len(sider_tasks), batch_size=50, mode='classification')
# Set nb_epoch=10 for better results.

for i in range(num_epochs):
    # Set nb_epoch=10 for better results.
    loss = model.fit(train_dataset, nb_epoch=1)
    print("Epoch %d loss: %f" % (i, loss))
    losses.append(loss)

Epoch 0 loss: 847.572515
Epoch 1 loss: 808.224588
Epoch 2 loss: 791.289381
Epoch 3 loss: 776.450479
Epoch 4 loss: 767.277516
Epoch 5 loss: 764.353805
Epoch 6 loss: 749.929250
Epoch 7 loss: 733.003933
Epoch 8 loss: 719.862713
Epoch 9 loss: 710.481849
Epoch 10 loss: 695.807742
Epoch 11 loss: 686.610951
Epoch 12 loss: 685.314363
Epoch 13 loss: 672.649579
Epoch 14 loss: 661.392732


In [11]:
metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

print("Evaluating model")
train_scores = model.evaluate(train_dataset, [metric], transformers)
print("Training ROC-AUC Score: %f" % train_scores["mean-roc_auc_score"])



Evaluating model
computed_metrics: [0.7898890942698706, 0.7122219192819825, 0.9917909747631111, 0.736620767390201, 0.7762178554099952, 0.7092531159488673, 0.8025601504488261, 0.8030320142924616, 0.73688064153721, 0.7971691149331956, 0.7515962036238135, 0.7763050280918853, 0.7999688497124913, 0.7754415920866697, 0.7287494678875817, 0.7845378484501768, 0.7934272300469483, 0.7755437239439449, 0.7032785050911938, 0.7176409945421468, 0.7569846463629499, 0.7456474592105702, 0.8488692774407061, 0.7316346076855325, 0.7477454652272895, 0.8324960127591707, 0.708001795828153]
Training ROC-AUC Score: 0.771611


In [12]:
valid_scores = model.evaluate(valid_dataset, [metric], transformers)
print("Validation ROC-AUC Score: %f" % valid_scores["mean-roc_auc_score"])

computed_metrics: [0.6821598101265822, 0.5603488920320603, 0.5, 0.59656137544982, 0.7175056348610067, 0.5205513784461153, 0.5913621262458472, 0.584406779661017, 0.5323581560283688, 0.6265249901613539, 0.5788288288288288, 0.5296143250688705, 0.5808823529411764, 0.5691056910569106, 0.5283185840707965, 0.684670551322277, 0.578248031496063, 0.6295862607338018, 0.5806976744186046, 0.617107393416082, 0.5739534883720931, 0.6191923577941816, 0.6908396946564885, 0.584251325989392, 0.6443298969072165, 0.7493638676844783, 0.5566922421142371]
Validation ROC-AUC Score: 0.600276


In [ ]:
# Hiperparameters permitidos pelo deepchem e os valores que são utilizados por defeito

In [29]:
params_dict = {"nb_epoch": [10],
              'learning_rate': [0.0005,0.5,0.75,1]}

In [30]:
len(train_dataset)

1141

In [31]:
n_features = len(train_dataset)

def model_builder(model_params, model_dir):
    model = GraphConvModel(
            len(sider_tasks), batch_size=50, mode='classification',**model_params)
    
    return model



In [32]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
optimizer = dc.hyper.HyperparamOpt(model_builder)
best_dnn, best_hyperparams, all_results = optimizer.hyperparam_search(
   params_dict, train_dataset, valid_dataset, [], metric)

Fitting model 1/4
hyperparameters: {'nb_epoch': 10, 'learning_rate': 0.0005}
computed_metrics: [0.6526898734177216, 0.5238095238095238, 0.8450704225352113, 0.5985605757696921, 0.6389932381667919, 0.47794486215538845, 0.55703211517165, 0.5542372881355933, 0.6779698581560284, 0.6168831168831169, 0.5253378378378379, 0.6163911845730028, 0.671218487394958, 0.6178861788617886, 0.5876106194690266, 0.6441057821604661, 0.6663385826771653, 0.6530054644808744, 0.5702325581395349, 0.4622234214786832, 0.5906976744186045, 0.5388623534520192, 0.5954198473282443, 0.47674418604651164, 0.6241595696996862, 0.688295165394402, 0.5846121057118498]
Model 1/4, Metric mean-roc_auc_score, Validation set 0: 0.602086
	best_validation_score so far: 0.602086
Fitting model 2/4
hyperparameters: {'nb_epoch': 10, 'learning_rate': 0.5}
computed_metrics: [0.5636372626582278, 0.5604667609618105, 0.630281690140845, 0.5015493802479009, 0.5632982719759579, 0.481140350877193, 0.5239479512735327, 0.5344915254237288, 0.43428634

In [36]:
best_dnn

GraphConvModel(dense_layer_size=128, dropout=[0.0, 0.0, 0.0],
               graph_conv_layers=[64, 64], mode='classification', n_classes=2,
               n_tasks=27, number_atom_features=75, uncertainty=False)

In [37]:
best_hyperparams

(10, 0.0005)

In [38]:
all_results

{'(10, 0.0005)': 0.6020863664194583,
 '(10, 0.5)': 0.5333155386285013,
 '(10, 0.75)': 0.530167447524502,
 '(10, 1)': 0.5525930040711647}

# Another Implementation GraphConvModel

TensorGraph - Simplicity is Beauty
TensorGraph is a simple, lean, and clean framework on TensorFlow for building any imaginable models.



What’s going on under the hood? 

Now we will build GraphConvModel ourselves.

The first step is to create a TensorGraph object. 
This object will hold the “computational graph” that defines the computation that a graph convolutional network will perform.


In [17]:
from deepchem.models.tensorgraph.tensor_graph import TensorGraph
from deepchem.models.tensorgraph.layers import Dense, GraphConv, BatchNorm
from deepchem.models.tensorgraph.layers import GraphPool, GraphGather

In [18]:
tg = TensorGraph(use_queue=False)

In [19]:
from deepchem.models.tensorgraph.layers import Feature

atom_features = Feature(shape=(None, 75))
degree_slice = Feature(shape=(None, 2), dtype=tf.int32)
membership = Feature(shape=(None,), dtype=tf.int32)

deg_adjs = []
for i in range(0, 10 + 1):
    deg_adj = Feature(shape=(None, i + 1), dtype=tf.int32)
    deg_adjs.append(deg_adj)

In [20]:
batch_size = 50

gc1 = GraphConv(64,activation_fn=tf.nn.relu,in_layers=[atom_features, degree_slice, membership] + deg_adjs)
batch_norm1 = BatchNorm(in_layers=[gc1])
gp1 = GraphPool(in_layers=[batch_norm1, degree_slice, membership] + deg_adjs)

gc2 = GraphConv(64,activation_fn=tf.nn.relu,in_layers=[gp1, degree_slice, membership] + deg_adjs)
batch_norm2 = BatchNorm(in_layers=[gc2])
gp2 = GraphPool(in_layers=[batch_norm2, degree_slice, membership] + deg_adjs)
    
dense = Dense(out_channels=128, activation_fn=tf.nn.relu, in_layers=[gp2])
batch_norm3 = BatchNorm(in_layers=[dense])

readout = GraphGather(batch_size=batch_size,activation_fn=tf.nn.tanh,in_layers=[batch_norm3, degree_slice, membership] + deg_adjs)

In [21]:
from deepchem.models.tensorgraph.layers import Dense, SoftMax, \
    SoftMaxCrossEntropy, WeightedError, Stack
from deepchem.models.tensorgraph.layers import Label, Weights

costs = []
labels = []
for task in range(len(sider_tasks)):
    classification = Dense(out_channels=2, activation_fn=None, in_layers=[readout])

    softmax = SoftMax(in_layers=[classification])
    tg.add_output(softmax)

    label = Label(shape=(None, 2))
    labels.append(label)
    cost = SoftMaxCrossEntropy(in_layers=[label, classification])
    costs.append(cost)
    
all_cost = Stack(in_layers=costs, axis=1)
weights = Weights(shape=(None, len(sider_tasks)))
loss = WeightedError(in_layers=[all_cost, weights])
tg.set_loss(loss)

In [22]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol

def data_generator(dataset, epochs=1, predict=False, pad_batches=True):
    for epoch in range(epochs):
        if not predict:
            print('Starting epoch %i' % epoch)
        for ind, (X_b, y_b, w_b, ids_b) in enumerate(
            dataset.iterbatches(batch_size, pad_batches=pad_batches, deterministic=True)):
            d = {}

            for index, label in enumerate(labels):
                d[label] = to_one_hot(y_b[:, index])
            d[weights] = w_b
            multiConvMol = ConvMol.agglomerate_mols(X_b)
            d[atom_features] = multiConvMol.get_atom_features()
            d[degree_slice] = multiConvMol.deg_slice
            d[membership] = multiConvMol.membership
            for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
                d[deg_adjs[i - 1]] = multiConvMol.get_deg_adjacency_lists()[i]
            yield d

In [23]:
# Epochs set to 1 to render tutorials online.
# Set epochs=10 for better results.
tg.fit_generator(data_generator(train_dataset, epochs=10))

Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9


772.023229184358

In [33]:
# Training dataset

In [24]:
metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

def reshape_y_pred(y_true, y_pred):
    """
    TensorGraph.Predict returns a list of arrays, one for each output
    We also have to remove the padding on the last batch
    Metrics taks results of shape (samples, n_task, prob_of_class)
    """
    n_samples = len(y_true)
    retval = np.stack(y_pred, axis=1)
    return retval[:n_samples]


print("Evaluating model")
train_predictions = tg.predict_on_generator(data_generator(train_dataset, predict=True))
train_predictions = reshape_y_pred(train_dataset.y, train_predictions)
train_scores = metric.compute_metric(train_dataset.y, train_predictions, train_dataset.w)

print("----------------")
print("Training ROC-AUC Score: %f" % train_scores)



Evaluating model
computed_metrics: [0.7551601971657425, 0.6855652250805043, 0.9932451449479313, 0.6951993619363386, 0.7224235807860262, 0.727182978970521, 0.7951928110277439, 0.7507341314660041, 0.68026188354705, 0.7510888019487709, 0.7108322221350694, 0.7084044520304389, 0.7479141935864084, 0.743953132359868, 0.6914698982872773, 0.7473882489078427, 0.762614282184334, 0.7114269928173188, 0.6658647570027227, 0.6770689306650496, 0.7146019724046404, 0.7004771278902939, 0.8067475638904211, 0.6684436149965817, 0.7106851729455828, 0.8235745614035088, 0.6718711147948612]
----------------
Training ROC-AUC Score: 0.734052


In [ ]:
# Validation Dataset

In [25]:
valid_predictions = tg.predict_on_generator(data_generator(valid_dataset, predict=True))
valid_predictions = reshape_y_pred(valid_dataset.y, valid_predictions)
valid_scores = metric.compute_metric(valid_dataset.y, valid_predictions, valid_dataset.w)
print("Valid ROC-AUC Score: %f" % valid_scores)


computed_metrics: [0.6748417721518988, 0.5363036303630363, 0.7253521126760563, 0.5841663334666134, 0.6705484598046582, 0.5010025062656642, 0.6146179401993355, 0.6518644067796611, 0.5839982269503545, 0.6334120425029516, 0.6005067567567568, 0.5020661157024793, 0.592436974789916, 0.5939024390243903, 0.5728613569321535, 0.6557597489914837, 0.6176181102362205, 0.6124121779859484, 0.6255813953488372, 0.595250944414463, 0.5946511627906976, 0.6615284411636996, 0.6590330788804071, 0.5359037127702977, 0.6295383236216943, 0.7131043256997456, 0.5598891730605285]
Valid ROC-AUC Score: 0.611043


In [ ]:
# Test Dataset

In [27]:
test_predictions = tg.predict_on_generator(data_generator(test_dataset, predict=True))
test_predictions = reshape_y_pred(test_dataset.y, test_predictions)
test_scores = metric.compute_metric(test_dataset.y, test_predictions, test_dataset.w)
print("Valid ROC-AUC Score: %f" % test_scores)

computed_metrics: [0.6493275316455696, 0.5961743393009378, 0.5283687943262412, 0.5894117647058823, 0.6663641863278887, 0.6321995464852609, 0.718011811023622, 0.542016806722689, 0.6634920634920635, 0.5993740219092332, 0.5237919132149902, 0.6000000000000001, 0.66, 0.42530345471521946, 0.6395568128241396, 0.5571025020177562, 0.6321195144724556, 0.638188608776844, 0.6961436170212767, 0.6845827439886846, 0.5972644376899696, 0.6195430436556507, 0.46111111111111114, 0.5506456820016142, 0.5884696016771489, 0.6979166666666667, 0.6515957446808511]
Valid ROC-AUC Score: 0.607707



# Comparision of Results with MoleculeNet results